# P4 Regularización, Normalización y Aumentado de datos



In [ ]:
## Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print('training set', x_train.shape)
print('val set', x_val.shape)

11490434/11490434 [==============================] - 0s 0us/step
training set (60000, 28, 28)
test set (10000, 28, 28)
training set (48000, 784)
val set (12000, 784)


## Modelo base
 Partiremos de una topología base e iremos añadiendo diferentes estrategias de regularización para mejorar el rendimiento del modelo.



In [ ]:
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/25
375/375 [==============================] - ETA: 0s - loss: 0.3477 - accuracy: 0.8989
Epoch 1: val_accuracy improved from -inf to 0.94850, saving model to best_model.h5


c:\Users\david\anaconda3\envs\vsc\lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


375/375 [==============================] - 41s 85ms/step - loss: 0.3477 - accuracy: 0.8989 - val_loss: 0.1809 - val_accuracy: 0.9485 - lr: 0.0250
Epoch 2/25
375/375 [==============================] - ETA: 0s - loss: 0.1332 - accuracy: 0.9613
Epoch 2: val_accuracy improved from 0.94850 to 0.96133, saving model to best_model.h5
375/375 [==============================] - 30s 81ms/step - loss: 0.1332 - accuracy: 0.9613 - val_loss: 0.1284 - val_accuracy: 0.9613 - lr: 0.0250
Epoch 3/25
375/375 [==============================] - ETA: 0s - loss: 0.0897 - accuracy: 0.9739
Epoch 3: val_accuracy improved from 0.96133 to 0.96958, saving model to best_model.h5
375/375 [==============================] - 30s 81ms/step - loss: 0.0897 - accuracy: 0.9739 - val_loss: 0.1015 - val_accuracy: 0.9696 - lr: 0.0250
Epoch 4/25
375/375 [==============================] - ETA: 0s - loss: 0.0647 - accuracy: 0.9811
Epoch 4: val_accuracy improved from 0.96958 to 0.97525, saving model to best_model.h5
375/375 [=======

KeyboardInterrupt: 

## Regularización l2 (o l1)

La regularización l2 consiste en añadir a la función de coste una penalización proporcional a la norma l2 de los pesos del modelo. De esta forma, se penaliza a los pesos que tengan un valor alto, forzando a que los pesos tengan valores pequeños. Esto se conoce como regularización l2. También podríamos hacer lo mismo con regularización l1 o con ambas (lo que se conoce como *Elastic net*)


In [ ]:
## Teniendo en cuenta el modelo base añade regularización L2 a las capas densas
from keras import Sequential
from keras.layers import Dense, Input
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(1024, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

## Dropout

El dropout es una técnica de regularización que consiste en eliminar aleatoriamente un porcentaje de las neuronas de la red durante el entrenamiento. De esta forma, se evita que la red se sobreajuste a los datos de entrenamiento y se mejora la generalización del modelo.


In [ ]:
## Teniendo en cuenta el modelo base añade regularización de tipo dropout a las capas densas
from keras import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
from keras.regularizers import l2

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


## Normalización BatchNorm

La normalización BatchNorm consiste en normalizar la salida de una capa de la red neuronal para que tenga media 0 y varianza 1. De esta forma, se consigue que la red neuronal pueda entrenarse más rápido y que sea más robusta a cambios en los pesos de las capas anteriores.


In [ ]:
## Teniendo en cuenta el modelo base añade normalización BatchNorm
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input(784))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


## Aumentado de datos

El aumentado de datos consiste en generar nuevos datos de entrenamiento a partir de los datos de entrenamiento originales. De esta forma, se consigue que el modelo sea más robusto y que se generalice mejor a datos que no ha visto durante el entrenamiento.

En nuestro caso para los dígitos de la MNIST vamos a realizar un aumento de datos de la siguiente forma:

- Rotación aleatoria de la imagen entre -30 y 30 grados.
- Traslación aleatoria de la imagen entre -3 y 3 píxeles en horizontal y vertical.
- Escalado aleatorio de la imagen entre 0.8 y 1.2.
- Inversión aleatoria de la imagen en horizontal y vertical. **NO!!!**

El aumentado de datos se ejecuta en CPU y ralentiza el entrenamiento.

Normalmente además, se necesitarán más epochs para entrenar el modelo.

In [ ]:
## Implementamos en el ejemplo base el aumentado de datos
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization,Reshape
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

## Importante: ImageDataGenerator espera una imagen con 3 canales, necesitamos hacer reshape
x_train = x_train.reshape(48000, 28, 28, 1)
x_val = x_val.reshape(12000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

## Ajustamos el generador de datos
datagen.fit(x_train)

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=SGD(learning_rate=0.025, momentum=0.9)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)


epochs=25
batch_size=128
## Entrenamos con el generador de datos en lugar de con el dataset
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_val, y_val),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])




## Ejercicio:

Probar todas las técnicas presentadas para obtener un acierto en **test > 99%**.

Se aconseja no malgastar datos de entrenamiento y por lo tanto emplear todo el training set para el entrenamiento. No emplear conjunto de validación y emplear el test set al final para calcular el acierto.

A modo de "trampa" podríamos ejecutar el fit con los datos de test en validation_data para así monitorizar si llegamos a ese 99%

validation_data=(x_test, y_test)


In [ ]:
from tensorflow import keras
from keras.datasets import mnist
from keras import Sequential
from keras.layers import Dense, Input, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau
##########################################################################Importar datos#########################################
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)
## Importante: ImageDataGenerator espera una imagen con 3 canales, necesitamos hacer reshape
x_train=x_train.reshape(60000, 28, 28, 1)
x_test=x_test.reshape(10000, 28, 28, 1)
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
print("Training set: ", x_train.shape)
print("Test set: ", x_test.shape)

####################################################################### Normalizar ################################################
x_train /= 255
x_test /= 255
# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

################################################################# Aumentado de datos  ##############################################
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')

## Ajustamos el generador de datos
datagen.fit(x_train)
##################################################################  MODELO  ##############################################
model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())#Normalizacion Batch,
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))
#optimizador
opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])
#callbacks Checkpoint y EarlyStop
#callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.01, patience=3) # con early stopping no llega a optimizar suficiente!!!
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,patience=2, min_lr=0.00001)
checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
#Numero de epochs
epochs = 100
batch_size = 128
#Entrenar el modelo, (no se usará conjunto de validación) Utilizar trampa: los datos de test utilizarlos como test del training
## Entrenamos con el generador de datos en lugar de con el dataset
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr, checkpoint])
#Probar el modelo con los test
## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)
Training set:  (60000, 28, 28, 1)
Test set:  (10000, 28, 28, 1)
Epoch 1/100
468/469 [============================>.] - ETA: 0s - loss: 0.6481 - accuracy: 0.8155
Epoch 1: val_accuracy improved from -inf to 0.95160, saving model to best_model.h5
469/469 [==============================] - 23s 44ms/step - loss: 0.6473 - accuracy: 0.8157 - val_loss: 0.1580 - val_accuracy: 0.9516 - lr: 0.0100
Epoch 2/100
469/469 [==============================] - ETA: 0s - loss: 0.3066 - accuracy: 0.9050
Epoch 2: val_accuracy did not improve from 0.95160
469/469 [==============================] - 20s 44ms/step - loss: 0.3066 - accuracy: 0.9050 - val_loss: 0.1694 - val_accuracy: 0.9445 - lr: 0.0100
Epoch 3/100
468/469 [============================>.] - ETA: 0s - loss: 0.2484 - accuracy: 0.9224
Epoch 3: val_accuracy improved from 0.95160 to 0.96600, saving model to best_model.h5
469/469 [==============================] - 21s 45ms/step - loss: 0.2481 - accuracy:

KeyboardInterrupt: ignored

In [ ]:
## Solución


# Importar y normalizar datos

from tensorflow import keras
from keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

print('training set', x_train.shape)
print('test set', x_test.shape)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# Normalize [0..255]-->[0..1]
x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
num_classes=10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

from keras import Sequential
from keras.layers import Dense, Input, BatchNormalization, Reshape
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode='nearest')



datagen.fit(x_train)

from keras import Sequential
from keras.layers import Dense, Input, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ReduceLROnPlateau

model = Sequential()

model.add(Input((28,28,1)))
model.add(Reshape((784,)))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

opt=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=2, min_lr=0.00001)

checkpoint = keras.callbacks.ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=50
batch_size=128
history = model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test),
                    callbacks=[reduce_lr,checkpoint])

## Cargar el mejor modelo y evaluarlo con el test set
model = keras.models.load_model('best_model.h5')
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

training set (60000, 28, 28, 1)
test set (10000, 28, 28, 1)
Epoch 1/50
